In [ ]:
from urllib.request import Request, urlopen
import requests
from bs4 import BeautifulSoup
import csv

In [ ]:
import pandas as pd
import numpy as np

## 1. Main page
### Extracting titles and url of respective single pages

In [ ]:
site = "http://www.bookdepository.com/bestsellers"
hdr = {'User-Agent': 'Mozilla/5.0'}
bookpage = requests.get(site)
soup = BeautifulSoup(bookpage.text, "html.parser")
print(soup.prettify())

In [ ]:
soup.h3

In [ ]:
soup.find_all('h3')

In [ ]:
titles = soup.find_all('h3', class_ ='title')
titles

In [ ]:
titles[0]

In [ ]:
len(titles)

In [ ]:
titles[0].text.strip()

In [ ]:
titles[0].a['href']

In [ ]:
type(titles)

In [ ]:
for i in titles:
    print(i.text.strip(), i.a['href'])

In [ ]:
title_db = []
url_db = []
for i in titles:
    name = i.text.strip()
    url = i.a['href']
    title_db.append(name)
    url_db.append(url)

In [ ]:
title_db

In [ ]:
len(title_db)

In [ ]:
url_db[0]

## 2. Single page
### Extracting specific information eg. author, price, bestseller rank, rating etc. about book

In [ ]:
nextsite = "http://www.bookdepository.com/Subtle-Art-Not-Giving-F-ck-Mark-Manson/9780062457714"
#nextsite = 'http://www.bookdepository.com'+ url_db[0]
#nextsite
hdr = {'User-Agent': 'Mozilla/5.0'}
onemoviepage = requests.get(nextsite)
onesoup = BeautifulSoup(onemoviepage.text, "html.parser")
print(onesoup.prettify())

In [ ]:
j = onesoup.find('ul', class_ = "biblio-info")
j

In [ ]:
j("span")[6].get_text().strip()

In [ ]:
b = onesoup.find('ol', class_ = "breadcrumb")
b

In [ ]:
b("a")[0].get_text().strip()

In [ ]:
b("a")[1].get_text().strip()

In [ ]:
onesoup.find('ul', class_ = "meta-info hidden-md").get_text()

In [ ]:
onesoup.find('div', class_ = "author-info hidden-md").get_text()

In [ ]:
onesoup.find_all("span")

In [ ]:
onesoup.find_all("span")[24].get_text()

In [ ]:
onesoup.find_all("span")[28].get_text()

In [ ]:
onesoup.find(itemprop="ratingValue").get_text()

In [ ]:
onesoup.find("span", class_="rating-count").get_text()

In [ ]:
onesoup.find("span", class_="sale-price").get_text()

In [ ]:
onesoup.find("span", class_="list-price").get_text()

In [ ]:
onesoup.find(itemprop="numberOfPages").get_text()

In [ ]:
onesoup.find(itemprop="datePublished").get_text()

In [ ]:
onesoup.find(itemprop="isbn").get_text()

### 2.1 Another single page
#### Choose any other single book page to check if biblio-info structure is always consistent across all single book pages (answer: it's not) 

In [ ]:
nextsite = "https://www.bookdepository.com/Thinking-Fast-Slow-Daniel-Kahneman/9780141033570"
#nextsite = url_db[0]
hdr = {'User-Agent': 'Mozilla/5.0'}
onemoviepage = requests.get(nextsite)
onesoup = BeautifulSoup(onemoviepage.text, "html.parser")

In [ ]:
s = onesoup.find('ul', class_ = "biblio-info")

In [ ]:
type(s)

In [ ]:
last_row = s("span")[-1].get_text().strip()
last_row

In [ ]:
country = s("span")[6].get_text().strip()
country

In [ ]:
onesoup.find('ul', class_ = "meta-info hidden-md").get_text().strip()

In [ ]:
onesoup.find('div', class_ = "author-info hidden-md").get_text().strip()

In [ ]:
onesoup.find_all("span")[24:30]

In [ ]:
onesoup.find("span", class_= "star full-star").parent

In [ ]:
onesoup.find(itemprop="ratingValue").get_text().strip()

In [ ]:
onesoup.find("span", class_="rating-count").get_text().strip()

In [ ]:
onesoup.find("span", class_="sale-price").get_text()

In [ ]:
onesoup.find("span", class_="list-price").get_text()

In [ ]:
onesoup.find(itemprop="numberOfPages").get_text()

In [ ]:
onesoup.find(itemprop="datePublished").get_text()

In [ ]:
onesoup.find(itemprop="isbn").get_text()

## 3. Putting everything together
#### In the event of a disconnection, re-run the code below for sites not yet crawled by updating the sitelist below

In [ ]:
sitelist = ['https://www.bookdepository.com/bestsellers?page='+str(i) for i in range(0,35)]
sitelist[0] = 'http://www.bookdepository.com/bestsellers'
#sitelist = ['https://www.bookdepository.com/bestsellers?page='+str(i) for i in range(15,35)]
sitelist

In [ ]:
sitelist[1]

In [ ]:
import time

In [ ]:
data = []
count = 0
for site in sitelist:
    hdr = {'User-Agent': 'Mozilla/5.0'}
    bookpage = requests.get(site)
    soup = BeautifulSoup(bookpage.text, "html.parser")
    #print(soup.prettify())
    titles = soup.find_all('h3', class_ ='title')
    count +=1
    for i in titles: 
        name = i.text.strip()
        nextsite = 'http://www.bookdepository.com' + i.a['href']
        hdr = {'User-Agent': 'Mozilla/5.0'}
        onebookpage = requests.get(nextsite)
        onesoup = BeautifulSoup(onebookpage.text, "html.parser")
        #print(onesoup.prettify())
        try:
            material = onesoup.find('ul', class_ = "meta-info hidden-md").get_text().strip()
        except: 
            material = "NA"
        try:
            author = onesoup.find('div', class_ = "author-info hidden-md").get_text().strip()
        except:
            author = "NA"
        s = onesoup.find('ul', class_ = "biblio-info")
        rank = s("span")[-1].get_text().strip()
        b = onesoup.find('ol', class_ = "breadcrumb")
        try:
            maincat = b("a")[0].get_text().strip()
        except:
            maincat = "NA"
        try:
            subcat = b("a")[1].get_text().strip()
        except:
            subcat = "NA"
        try:
            rating = onesoup.find(itemprop="ratingValue").get_text().strip()
        except:
            rating = "NA"
        try:
            ratingcount = onesoup.find("span", class_="rating-count").get_text().strip()
        except:
            ratingcount = "NA"
        try: 
            saleprice = onesoup.find("span", class_="sale-price").get_text()
        except:
            saleprice = "NA"
        try:
            listprice = onesoup.find("span", class_="list-price").get_text()
        except:
            listprice = "NA"
        try:
            numofpages = onesoup.find(itemprop="numberOfPages").get_text()
        except:
            numofpages = "NA"
        try:
            datepub = onesoup.find(itemprop="datePublished").get_text()
        except:
            datepub = "NA"
        try:
            isbn13 = onesoup.find(itemprop="isbn").get_text()
        except:
            isbn13 = "NA"
        data.append((name,material,author,rank,maincat,subcat,rating,ratingcount,saleprice,
                    listprice,numofpages,datepub,isbn13))    
    print(count)
    time.sleep(1)

In [ ]:
len(data)

In [ ]:
df = pd.DataFrame(np.array(data))

In [ ]:
df.columns = ['name','material','author','rank','maincat','subcat','rating','ratingcount','saleprice',
                    'listprice','numofpages','datepub','isbn13']

In [ ]:
df.head()

In [ ]:
df.to_csv('bookdepo.csv')
#df.to_csv('bookdepo1.csv')

#### Resources: 
http://altitudelabs.com/blog/web-scraping-with-python-and-beautiful-soup/
http://web.stanford.edu/~zlotnick/TextAsData/Web_Scraping_with_Beautiful_Soup.html  
https://www.crummy.com/software/BeautifulSoup/bs4/doc/  
https://www.dataquest.io/blog/web-scraping-beautifulsoup/